In [1]:
import json
import pickle
import re
import collections
import pandas as pd
pd.set_option('display.max_rows', 100)
from os import listdir
from os.path import isfile, join
import numpy as np
import sys
import math
import time
import sklearn.feature_selection as fs

# find representatives

In [2]:
rep_path = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\\CS597\\twitterMining\\InitialSet\\Football\\location\\rep\\'
path = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\\CS597\\twitterMining\\InitialSet\\Football\\location\\test1\\'
with open (rep_path + 'followees_list.pickle','rb+') as f:
    followees_list = pickle.load(f)

In [3]:
len(followees_list)

83

In [4]:
def find_common_followee(followees_list):
    
    followee_vocab = {}

    for user in followees_list.keys():
        counter = collections.Counter(followees_list[user])
       
        for c in counter.keys():
            if c not in followee_vocab.keys():
                followee_vocab[c] = counter[c]
            else:
                followee_vocab[c] += counter[c]
    
    #ranked_followees = sorted(followee_vocab.items(),key=lambda x:x[1],reverse=True) #[:300]
    
    # delete those only one followers nodes
    
    for node in list(followee_vocab.keys()):
        if followee_vocab[node] == 1 or followee_vocab[node] == 2:
            del followee_vocab[node]
    
    return followee_vocab

# countervectorize write by myself!

# complete followees in the inital set (raw)

In [5]:
path_test2 = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\\CS597\\twitterMining\\InitialSet\\Football\\location\\test2\\'

In [6]:
with open(path_test2 + 'de_complete_followees.pickle','rb+') as f:
    details = pickle.load(f)

In [8]:
df_raw = pd.DataFrame(details, columns = ['id','screen_name','location','counts','total_followers'])
print(df_raw.shape)
df_raw.head()

(1590, 5)


,id,screen_name,location,counts,total_followers
0,50537131,rcobb18,"Titletown, USA x Alcoa,TN",4,483970
1,582348811,Drunk_Kane88,PROBABLY WITH YOUR SISTER,6,131334
2,4207961,chrislhayes,Brooklyn,5,750134
3,128102424,chanjones55,607/AZ,3,246445
4,90570778,ZachBoychuk,"Calgary, Raleigh, Charlotte",4,460803


# leverage 

too many features 

<b>combine Amin's approach rank these</b>

Rank: (count of following in the initial list)/(count of followers of that rep node)

In [11]:
df_raw['IDF'] = df_raw['counts']/df_raw['total_followers']
df_ranked = df_raw.sort_values(by='IDF',ascending =False)
# average twitter followers number is 208, how do we set this cut off
df_ranked = df_ranked[df_ranked['total_followers']>1000]
df_ranked = df_ranked[df_ranked['counts']>3]

In [13]:
df_ranked.head(100)

,id,screen_name,location,counts,total_followers,IDF
1413,65244726,FulphilO,Connecticut,4,2362,0.001693
774,1250219924,Johnathan_Wood1,,4,2701,0.001481
816,337439061,charleslenojr72,"Chicago, IL",5,3466,0.001443
810,1042511856,TedBear34,Flying over Soldier Field,4,3134,0.001276
80,96526542,keribo1,Me: Chicago | My heart: the UK,4,3598,0.001112
183,1018823131,BeardedBearsMan,Melb Australia,4,3798,0.001053
678,635721381,AaronLemingNFL,Southern California,9,8898,0.001011
1286,2162089176,Dylan120Sports,Chicago,6,5996,0.001001
344,222249852,CEmma670,Chicago,8,8870,0.000902
15,231907379,Dbrave_8,,4,5266,0.000760


modify here: divided by (common followers not in the target group) = /(1 - p(target))*common followers
<p>query each representative to sample some of their followers, in order to get a distribution of how many of their followers are with in the target.
<p>right now only user location, in the future can put semantic approach into this.

In [15]:
df_ranked['Tmax'] = df_ranked['total_followers']/(df_ranked['counts']/len(followees_list))

# probability of user in the inital set following this rep
#df_ranked['I_Vx=1'] = df_ranked['counts']/len(followees_list)
# probability of general user set following this rep
# assuming twitter has a total of 883000000 users
#df_ranked['Vx=1'] = df_ranked['total_followers']/223000000 in US 60 million

#p(follow|T)/P(follow)

constant = 60000000/len(followees_list)

df_ranked['follow'] = df_ranked['counts']/df_ranked['total_followers']* constant
df_ranked['Vxy_1'] = df_ranked['counts']/len(followees_list)
#p(not_follow|T)/p(not_follow)
df_ranked['unfollow'] = (len(followees_list)-df_ranked['counts'])/(60000000-df_ranked['total_followers']) * constant
df_ranked['Vxy_0'] = 1-df_ranked['Vxy_1']

df_ranked.head(30)

,id,screen_name,location,counts,total_followers,IDF,Tmax,follow,Vxy_1,unfollow,Vxy_0
1413,65244726,FulphilO,Connecticut,4,2362,0.001693,49011.500000,1224.202483,0.048193,0.951845,0.951807
774,1250219924,Johnathan_Wood1,,4,2701,0.001481,56045.750000,1070.553967,0.048193,0.951850,0.951807
816,337439061,charleslenojr72,"Chicago, IL",5,3466,0.001443,57535.600000,1042.832611,0.060241,0.939813,0.939759
810,1042511856,TedBear34,Flying over Soldier Field,4,3134,0.001276,65030.500000,922.643990,0.048193,0.951857,0.951807
80,96526542,keribo1,Me: Chicago | My heart: the UK,4,3598,0.001112,74658.500000,803.659329,0.048193,0.951864,0.951807
183,1018823131,BeardedBearsMan,Melb Australia,4,3798,0.001053,78808.500000,761.339196,0.048193,0.951867,0.951807
678,635721381,AaronLemingNFL,Southern California,9,8898,0.001011,82059.333333,731.178253,0.108434,0.891699,0.891566
1286,2162089176,Dylan120Sports,Chicago,6,5996,0.001001,82944.666667,723.373815,0.072289,0.927804,0.927711
344,222249852,CEmma670,Chicago,8,8870,0.000902,92026.250000,651.987884,0.096386,0.903748,0.903614
15,231907379,Dbrave_8,,4,5266,0.000760,109269.500000,549.101076,0.048193,0.951891,0.951807
